## [24 式加速你的 Python -  梁云1991](https://mp.weixin.qq.com/s/Meb-XCHj0Yj73KyMxw2Pyw)

- 有些方法适用于大规模数据，在小数据集情况下，效果反而不好
- %%time 似乎与列表推导式有冲突，两者在同一单元格内，可能会报错
- 有两个库，似乎只支持 Linux 和 OS 系统

In [21]:
import time
import numpy as np
import pandas as pd

## 一、分析代码运行时间

### 第1式 测算代码运行时间

平凡方法

In [1]:
import time 

tic = time.time()
much_job = [x**2 for x in range(1, 1000000, 3)]
toc = time.time()
print("used {:.5}s".format(toc - tic))

used 0.10689s


快捷方法

In [2]:
%%time

much_job = [x**2 for x in range(1, 1000000, 3)]

Wall time: 107 ms


### 第2式 测算代码多次运行平均时间

平凡方法

In [4]:
from timeit import timeit

g = lambda x: x**2 + 1
def main():
    return (g(2)**120)

timeit("main()", globals = {"main": main}, number=10)

1.1199999960354035e-05

快捷方法

In [5]:
%%timeit -n 10

g = lambda x: x**2 + 1
def main():
    return (g(2)**120)
main()

1.38 µs ± 828 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 第3式 按调用函数分析代码运行时间

平凡方法

In [3]:
def relu(x):
    return (x if x > 0 else 0)
def main():
    result = [relu(x) for x in range(-100000, 100000, 1)]
    return result

In [4]:
import profile

profile.run("main()")

         200006 function calls in 0.453 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.453    0.453 :0(exec)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
   200000    0.203    0.000    0.203    0.000 <ipython-input-3-a72872bc80f3>:1(relu)
        1    0.000    0.000    0.453    0.453 <ipython-input-3-a72872bc80f3>:3(main)
        1    0.250    0.250    0.453    0.453 <ipython-input-3-a72872bc80f3>:4(<listcomp>)
        1    0.000    0.000    0.453    0.453 <string>:1(<module>)
        1    0.000    0.000    0.453    0.453 profile:0(main())
        0    0.000             0.000          profile:0(profiler)




快捷方法

In [5]:
%prun main()

#### 第4式 按行分析代码运行时间

平凡方法

快捷方法

## 二、加速你的查找

### 第5式 用 set 而非 list 进行查找

低速方法

In [31]:
# data1 and data2 均为迭代器，只能用一次，即只要 list 迭代到末尾后，set 就不能再次迭代

data1 = (i**2 + 1 for i in range(1000000))
data2 = (i**2 + 1 for i in range(1000000))
list_data = list(data1)
set_data = set(data2)

In [32]:
%%time
1098987 in list_data

Wall time: 12 ms


False

In [33]:
%%time
1098987 in set_data

Wall time: 0 ns


False

### 第6式 用 dict 而非两个 list 进行匹配查找

低速方法

In [34]:
list_a = [2*i-1 for i in range(1000000)]
list_b = [i**2 for i in list_a]
dict_ab = dict(zip(list_a, list_b))

In [35]:
%%time
print(list_b[list_a.index(876567)])

768369705489
Wall time: 6 ms


高速方法

In [36]:
%%time
print(dict_ab.get(876567, None))

768369705489
Wall time: 0 ns


## 三、加速你的循环

### 第7式 优先使用 for 循环而不是 while 循环

低速方法

In [37]:
%%time
s,i = 0, 0
while i < 10000:
    i = i + 1
    s = s + 1
print(s)

10000
Wall time: 2.03 ms


高速方法

In [38]:
%%time
s = 0
for i in range(1, 10001):
    s = s + i
print(s)

50005000
Wall time: 1.03 ms


### 第8式 在循环体中避免重复计算

In [5]:
a = [i**2+1 for i in range(2000)]
sum_a = sum(a)

低速方法

In [6]:
%%time
b = [i / sum(a) for i in a]

Wall time: 30.9 ms


In [7]:
%%time
b = [i / sum_a for i in a]

Wall time: 0 ns


## 四、加速你的函数

### 第9式 用循环机制代替递归函数

低速方法

In [14]:
%%time
def fib(n):
    return ( 1 if n in (1, 2) else fib(n-1)+fib(n-2))
fib(30)

Wall time: 230 ms


高速方法

In [15]:
%%time
def fib(n):
    if n in (1, 2):
        return 1
    a, b = 1, 1
    for i in range(2, n):
        a, b = b, a+b
    return b
fib(30)

Wall time: 0 ns


### 第10式 用缓存机制递归函数

低速方法

In [17]:
%%time
def fib(n):
    return (1 if n in (1,2) else fib(n-1)+fib(n-2))
fib(30)

Wall time: 226 ms


高速方法

In [27]:
%%time

from functools import lru_cache
@lru_cache(100)
def fib(n):
    return (1 if n in (1, 2) else fib(n-1)+fib(n-2))
fib(30)

Wall time: 244 ms


### 第11式 用 numba 加速 Python 函数

低速方法

In [26]:
%%time

def my_power(x):
    return x**2

def my_power_sum(n):
    s = 0
    for i in range(1, n+1):
        s = s + my_power(i)
    return s

print(my_power_sum(1000000))

333333833333500000
Wall time: 492 ms


高速方法

In [33]:
%%time

from numba import jit

@jit
def my_power(x):
    return x**2

@jit
def my_power_sum(n):
    s = 0
    for i in range(1, n+1):
        s = s + my_power(i)
    return s

print(my_power_sum(1000000))

333333833333500000
Wall time: 682 ms


## 五、使用标准库函数进行加速

### 第12式 使用 collections.Counter 加速计数

低速方法

In [34]:
data = [x**2%1989 for x in range(2000000)]

In [36]:
%%time

values_count = {}
for i in data:
    i_cnt = values_count.get(i, 0)
    values_count[i] = i_cnt + 1
print(values_count.get(4, 0))

8044
Wall time: 433 ms


高速方法

In [37]:
%%time 

from collections import Counter

values_count = Counter(data)
print(values_count.get(4, 0))

8044
Wall time: 130 ms


### 第13式 使用 collections.ChainMap 加速字典合并

In [40]:
dic_a = {i: i+1 for i in range(1, 1000000, 2)}
dic_b = {i: 2*i+1 for i in range(1, 1000000, 3)}
dic_c = {i: 3*i+1 for i in range(1, 1000000, 5)}
dic_d = {i: 4*i+1 for i in range(1, 1000000, 7)}

低速方法

In [41]:
%%time

result = dic_a.copy()
result.update(dic_b)
result.update(dic_c)
result.update(dic_d)
print(result.get(9999, 0))

10000
Wall time: 81.8 ms


高速方法

In [42]:
%%time

from collections import ChainMap

chain = ChainMap(dic_a, dic_b, dic_c, dic_d)
print(chain.get(9999, 0))

10000
Wall time: 0 ns


## 六、使用高阶函数进行加速

### 第14式 使用 map 代替推导式进行加速

低速方法

In [43]:
%%time

result = [x**2 for x in range(1, 1000000, 3)]

Wall time: 108 ms


高速方法

In [46]:
%%time

result = map(lambda x: x**2, range(1, 1000000, 3))

Wall time: 0 ns


### 第15式 使用 filter 代替推导式进行加速

低速方法

In [48]:
%%time

result = [x for x in range(1, 1000000, 3) if x%7 == 0]

Wall time: 21 ms


高速方法

In [49]:
%%time

result = filter(lambda x: x%7==0, range(1, 1000000, 3))

Wall time: 0 ns


## 七、使用 numpy 向量化进行加速

### 第16式 使用 np.array 代替 list

低速方法

In [15]:
%%time

a = range(1, 1000000, 3)
b = range(1000000, 1, -3)
c = [3*a[i]-2*b[i] for i in range(0, len(a))]

Wall time: 108 ms


高速方法

In [7]:
%%time

import numpy as np

array_a = np.arange(1, 1000000, 3)
array_b = np.arange(1000000, 1, -3)
array_c = 3 * array_a - 2 * array_b

Wall time: 4.99 ms


### 使用17式 使用 np.ufunc 代替 math.func?

低速方法

In [13]:
%%time

import math

a = range(1, 1000000, 3)
b = [math.log(x) for x in a]

Wall time: 56 ms


高速方法

In [16]:
%%time

import numpy as np

array_a = np.arange(1, 1000000, 3)
array_b = np.log(array_a)

Wall time: 6.02 ms


### 第18式 使用 np.where 代替 if

In [18]:
array_a = np.arange(-100000, 1000000)

低速方法

In [19]:
%%time
# np.vectorize 可以将普通函数转换成支持向量化的函数

relu = np.vectorize(lambda x: x if x>0 else 0)
array_b = relu(array_a)

Wall time: 143 ms


高速方法

In [20]:
%%time

relu = lambda x: np.where(x >0, x, 0)
array_b = relu(array_a)

Wall time: 6.53 ms


## 八、加速你的 Pandas

### 第19式 使用 csv 文件读写代替 excel 文件读写

In [23]:
data = {'province': ['SiChuan', 'YuanNan', 'GuangDong'], 'capital': ['chengdu', 'kunming', 'guangzhou']}
df = pd.DataFrame(data)

低速方法

In [25]:
%%time

df.to_excel("data.xlsx")

Wall time: 384 ms


高速方法

In [26]:
%%time

df.to_csv("data.csv")

Wall time: 2.98 ms


### 第20式 使用 pandas 多进程工具 pandarallel

In [28]:
df = pd.DataFrame(np.random.randint(-10, 11, size=(10000, 26)),
                 columns=list("abcdefghijklmnopqrstuvwxyz"))

低速方法

In [34]:
%%time

result = df.apply(np.sum, axis=1)

Wall time: 750 ms


高速方法

In [37]:
%%time

# from pandarallel import pandarallel
# 不支持 windows

# pandarallel.initialize(nb_workers=4)
# result = df.parallel_apply(np.sum, axis=1)

Wall time: 0 ns


## 九、使用 Dask 进行加速

### 第21式 使用 dask 加速 dataframe

In [47]:
df = pd.DataFrame(np.random.randint(0, 6, size=(100000000, 5)), 
                 columns = list('abcde'))

低速方法

In [48]:
%%time

df.groupby('a').mean()

Wall time: 17.5 s


,b,c,d,e
a,,,,
0,2.500011,2.500534,2.500203,2.500017
1,2.500034,2.499888,2.500185,2.500545
2,2.499706,2.500044,2.499576,2.499715
3,2.499983,2.500159,2.499770,2.499656
4,2.500130,2.500217,2.500006,2.499874
5,2.500024,2.499669,2.500411,2.500110


高速方法

In [49]:
import dask.dataframe as dd

df_dask = dd.from_pandas(df, npartitions=40)
%time df_dask.groupby("a").mean().compute()

Wall time: 6.61 s


,b,c,d,e
a,,,,
0,2.500011,2.500534,2.500203,2.500017
1,2.500034,2.499888,2.500185,2.500545
2,2.499706,2.500044,2.499576,2.499715
3,2.499983,2.500159,2.499770,2.499656
4,2.500130,2.500217,2.500006,2.499874
5,2.500024,2.499669,2.500411,2.500110


### 第22 式 使用 dask.delayed 进行加速

低速方法

In [50]:
def muchjob(x):
    time.sleep(5)
    return x**2

In [51]:
%%time

result = [muchjob(i) for i in range(5)]
result

Wall time: 25 s


高速方法

In [54]:
%%time

from dask import delayed, compute
from dask import threaded, multiprocessing

values = [delayed(muchjob)(i) for i in range(5)]
result = compute(*values, scheduler='multiprocessing')

Wall time: 5.61 s


## 十、应用多线程多进程加速

### 第23式 应用多线程加速 IO 密集型任务

低速方法

In [56]:
%%time

def writefile(i):
    with open(str(i) + ".txt", "w") as f:
        s = ("hello %d" %i) * 10000000
        f.write(s)

# 串行任务
for i in range(10):
    writefile(i)

Wall time: 6.07 s


高速方法

In [61]:
# %%time

# import threading

# def writefile(i):
#     with open(str(i) + ".txt", "w") as f:
#         s = ("hello %d" %i) * 10000000
#         f.write(s)
        
# # 多线程任务
# thread_list = []
# for i in range(10):
#     t = threading.Thread(target=writefile, args=(i, ))
#     t.setDaemon(True)  # 设置为守护线程
#     thread_list.append(t)
    
# for t in thread_list:
#     t.start()  # 启动线程
    
# for t in thread_list:
#     t.join()  # 等待子线程结束

### 第24式 应用多进程加速 CPU 密集型任务

低速方法

In [63]:
%%time

def muchjob(x):
    time.sleep(5)
    return x**2
    
# 串行任务
ans = [muchjob(i) for i in range(8)]
print(ans)

[0, 1, 4, 9, 16, 25, 36, 49]
Wall time: 40 s


高速方法

In [ ]:
# %%time

import multiprocessing

data = range(8)

def muchjob(x):
    time.sleep(5)
    return x**2

# 多进程任务
pool = multiprocessing.Pool(processes=4)
result = []
for i in range(8):
    result.append(pool.apply_async(muchjob, (i, )))
pool.close()
pool.join()
ans = [res.get() for res in result]
print(ans)